# Business Understanding

Option #1

Kickstarter wants to help make more projects successful (meet funding goal) because when projects are successful Kickstarter gets a cut of the funding. We rank the importance of features to get an understanding of what makes a project successful.  How much does staff_pick or spotlight influence the success? Time of launch-deadline? Category listings? Should we market ourselves to xxx, who is the most successful?

Option #2 

We have an idea we want to kickstart, is it critical that we have good rewards/get spotlight/get backers/ how long should the deadline be etc to hit a successful funding goal.



# Imports

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
pd.set_option('max_columns', None)

# EDA

In [2]:
# df = pd.DataFrame()
# for f in glob('../data/Kickstarter*.csv'):
#     df = pd.concat([df, pd.read_csv(f)])
# df.to_csv('..data/Kickstarter_full.csv', index = False)

In [3]:
df = pd.read_csv('../data/Kickstarter_full.csv')

In [4]:
df.head()

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,launched_at,location,name,permissions,photo,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,2,I am building a website as a platform for stud...,"{""id"":362,""name"":""Makerspaces"",""slug"":""technol...",8,AU,Australia,1526014096,"{""id"":576665900,""name"":""Emma - Teaching Little...",AUD,$,True,USD,1528928562,False,NaN,0.713110,1000.0,2116213542,NaN,False,NaN,1526336562,"{""id"":1102170,""name"":""Geelong"",""slug"":""geelong...",STEAM Challenges for kids,NaN,"{""key"":""assets/021/192/015/169fac7e245d13c5b2c...",11.32,"{""id"":3378817,""project_id"":3378817,""state"":""in...",steam-challenges-for-kids,https://www.kickstarter.com/discover/categorie...,False,False,failed,1528928562,0.754091,"{""web"":{""project"":""https://www.kickstarter.com...",8.536309,domestic
1,43,"For my first solo museum exhibition, I'm const...","{""id"":288,""name"":""Installations"",""slug"":""art/i...",3390,US,the United States,1490746168,"{""id"":1570512397,""name"":""Mark Reamy"",""is_regis...",USD,$,True,USD,1493819063,False,NaN,1.000000,3000.0,162726099,NaN,False,NaN,1491227063,"{""id"":1123319,""name"":""Gwangju-Si"",""slug"":""gwan...",Moving Mountains - An Exhibition in South Korea,NaN,"{""key"":""assets/016/043/280/779386fd82d6830e626...",3390.00,"{""id"":2940021,""project_id"":2940021,""state"":""in...",moving-mountains-an-exhibition-in-south-korea,https://www.kickstarter.com/discover/categorie...,True,True,successful,1493819064,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",3390.000000,domestic
2,243,"Jenn lives in an underground bunker, protected...","{""id"":302,""name"":""Thrillers"",""slug"":""film & vi...",28601,US,the United States,1411536270,"{""id"":579435811,""name"":""Steve Desmond"",""is_reg...",USD,$,True,USD,1415844585,False,NaN,1.000000,16500.0,1678056872,NaN,False,NaN,1413248985,"{""id"":2442047,""name"":""Los Angeles"",""slug"":""los...",Monsters - The Short Film,NaN,"{""key"":""assets/011/877/406/34b5aa93986724b6374...",28601.11,"{""id"":1376329,""project_id"":1376329,""state"":""in...",monsters-the-short-film,https://www.kickstarter.com/discover/categorie...,True,True,successful,1415844587,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",28601.110000,domestic
3,2,I want to create a book that displays my work ...,"{""id"":278,""name"":""People"",""slug"":""photography/...",26,US,the United States,1420492380,"{""id"":528048907,""name"":""Brandi Carroll"",""is_re...",USD,$,True,USD,1423192306,False,NaN,1.000000,5000.0,1972792590,NaN,False,NaN,1420600306,"{""id"":2374330,""name"":""Caneyville"",""slug"":""cane...",Imagine,NaN,"{""key"":""assets/011/991/354/8497459c8c1bef38223...",26.00,"{""id"":1612632,""project_id"":1612632,""state"":""in...",imagine-0,https://www.kickstarter.com/discover/categorie...,False,False,failed,1423192306,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",26.000000,domestic
4,210,A series of hard enamel pins and accessories,"{""id"":26,""name"":""Crafts"",""slug"":""crafts"",""posi...",6516,GB,the United Kingdom,1593683203,"{""id"":926607047,""name"":""Laura Croft"",""slug"":""f...",GBP,£,False,USD,1598633946,False,NaN,1.301187,1000.0,928655800,NaN,False,NaN,1596214746,"{""id"":26279,""name"":""Lincoln"",""slug"":""lincoln-g...",Monsters and Maw Enamel Pins,NaN,"{""key"":""assets/029/762/429/cfde26eb92bacd30e93...",4919.00,"{""id"":4025596,""project_id"":4025596,""state"":""in...",monsters-and-maw-enamel-pins,https://www.kickstarter.com/discover/categorie...,True,False,successful,1598633946,1.298425,"{""web"":{""project"":""https://www.kickstarter.com...",6386.951345,domestic


In [5]:
df.state.value_counts(normalize = True)

successful    0.582305
failed        0.344952
canceled      0.041410
live          0.031334
Name: state, dtype: float64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219538 entries, 0 to 219537
Data columns (total 38 columns):
backers_count               219538 non-null int64
blurb                       219532 non-null object
category                    219538 non-null object
converted_pledged_amount    219538 non-null int64
country                     219538 non-null object
country_displayable_name    219538 non-null object
created_at                  219538 non-null int64
creator                     219538 non-null object
currency                    219538 non-null object
currency_symbol             219538 non-null object
currency_trailing_code      219538 non-null bool
current_currency            219538 non-null object
deadline                    219538 non-null int64
disable_communication       219538 non-null bool
friends                     312 non-null object
fx_rate                     219538 non-null float64
goal                        219538 non-null float64
id                          219

# Data Cleaning

In [7]:
country_names = list(df.country_displayable_name.value_counts().index)
country_codes = list(df.country.value_counts().index)
countries = list(zip(country_names, country_codes))

From the .info we see a few columns that can be dropped because they don't have enough data

- `friend`
- `is_backing`
- `is_starred`
- `permission`

Additionally, there are a few unneeded or redundant columns
- `photo`
- `id`
- `currency_symbol`
- `country_displayable_name`
- `disable_communication`
- `fx_rate`
- `converted_pledged_amount`
- `currency_trailing_code`
- `current_currency`

In [8]:
df = df.drop(columns = ['friends',
                        'is_backing',
                        'is_starred',
                        'permissions',
                        'photo',
                        'id',
                        'currency_symbol',
                        'country_displayable_name',
                        'disable_communication',
                        'converted_pledged_amount',
                        'fx_rate',
                        'currency_trailing_code',
                        'current_currency'
                       ]
            )

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219538 entries, 0 to 219537
Data columns (total 25 columns):
backers_count       219538 non-null int64
blurb               219532 non-null object
category            219538 non-null object
country             219538 non-null object
created_at          219538 non-null int64
creator             219538 non-null object
currency            219538 non-null object
deadline            219538 non-null int64
goal                219538 non-null float64
is_starrable        219538 non-null bool
launched_at         219538 non-null int64
location            219332 non-null object
name                219538 non-null object
pledged             219538 non-null float64
profile             219538 non-null object
slug                219538 non-null object
source_url          219538 non-null object
spotlight           219538 non-null bool
staff_pick          219538 non-null bool
state               219538 non-null object
state_changed_at    219538 non-null i

To deal with potential duplicate entries, rows with the same `blurb` and `launched_at` timestamp are dropped

In [10]:
df = df.drop_duplicates(subset = ['blurb', 'launched_at'])

We need to convert the date columns with object dtype unix timestamps into datetime objects.

In [11]:
df.created_at = df.created_at.apply(lambda x: dt.fromtimestamp(x))
df.launched_at = df.launched_at.apply(lambda x: dt.fromtimestamp(x))
df.deadline = df.deadline.apply(lambda x: dt.fromtimestamp(x))
df.state_changed_at = df.state_changed_at.apply(lambda x: dt.fromtimestamp(x))

To convert the `goal` feature into a consistent currency, the `usd_goal` is created using `static_usd_rate`

In [12]:
df['usd_goal'] = df.static_usd_rate * df.goal

In [13]:
df.head()

,backers_count,blurb,category,country,created_at,creator,currency,deadline,goal,is_starrable,launched_at,location,name,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,usd_goal
0,2,I am building a website as a platform for stud...,"{""id"":362,""name"":""Makerspaces"",""slug"":""technol...",AU,2018-05-11 00:48:16,"{""id"":576665900,""name"":""Emma - Teaching Little...",AUD,2018-06-13 18:22:42,1000.0,False,2018-05-14 18:22:42,"{""id"":1102170,""name"":""Geelong"",""slug"":""geelong...",STEAM Challenges for kids,11.32,"{""id"":3378817,""project_id"":3378817,""state"":""in...",steam-challenges-for-kids,https://www.kickstarter.com/discover/categorie...,False,False,failed,2018-06-13 18:22:42,0.754091,"{""web"":{""project"":""https://www.kickstarter.com...",8.536309,domestic,754.09094
1,43,"For my first solo museum exhibition, I'm const...","{""id"":288,""name"":""Installations"",""slug"":""art/i...",US,2017-03-28 20:09:28,"{""id"":1570512397,""name"":""Mark Reamy"",""is_regis...",USD,2017-05-03 09:44:23,3000.0,False,2017-04-03 09:44:23,"{""id"":1123319,""name"":""Gwangju-Si"",""slug"":""gwan...",Moving Mountains - An Exhibition in South Korea,3390.00,"{""id"":2940021,""project_id"":2940021,""state"":""in...",moving-mountains-an-exhibition-in-south-korea,https://www.kickstarter.com/discover/categorie...,True,True,successful,2017-05-03 09:44:24,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",3390.000000,domestic,3000.00000
2,243,"Jenn lives in an underground bunker, protected...","{""id"":302,""name"":""Thrillers"",""slug"":""film & vi...",US,2014-09-24 01:24:30,"{""id"":579435811,""name"":""Steve Desmond"",""is_reg...",USD,2014-11-12 21:09:45,16500.0,False,2014-10-13 21:09:45,"{""id"":2442047,""name"":""Los Angeles"",""slug"":""los...",Monsters - The Short Film,28601.11,"{""id"":1376329,""project_id"":1376329,""state"":""in...",monsters-the-short-film,https://www.kickstarter.com/discover/categorie...,True,True,successful,2014-11-12 21:09:47,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",28601.110000,domestic,16500.00000
3,2,I want to create a book that displays my work ...,"{""id"":278,""name"":""People"",""slug"":""photography/...",US,2015-01-05 16:13:00,"{""id"":528048907,""name"":""Brandi Carroll"",""is_re...",USD,2015-02-05 22:11:46,5000.0,False,2015-01-06 22:11:46,"{""id"":2374330,""name"":""Caneyville"",""slug"":""cane...",Imagine,26.00,"{""id"":1612632,""project_id"":1612632,""state"":""in...",imagine-0,https://www.kickstarter.com/discover/categorie...,False,False,failed,2015-02-05 22:11:46,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",26.000000,domestic,5000.00000
4,210,A series of hard enamel pins and accessories,"{""id"":26,""name"":""Crafts"",""slug"":""crafts"",""posi...",GB,2020-07-02 05:46:43,"{""id"":926607047,""name"":""Laura Croft"",""slug"":""f...",GBP,2020-08-28 12:59:06,1000.0,False,2020-07-31 12:59:06,"{""id"":26279,""name"":""Lincoln"",""slug"":""lincoln-g...",Monsters and Maw Enamel Pins,4919.00,"{""id"":4025596,""project_id"":4025596,""state"":""in...",monsters-and-maw-enamel-pins,https://www.kickstarter.com/discover/categorie...,True,False,successful,2020-08-28 12:59:06,1.298425,"{""web"":{""project"":""https://www.kickstarter.com...",6386.951345,domestic,1298.42475


In [14]:
df.currency.value_counts()

USD    133128
GBP     21420
EUR     14356
CAD      9035
AUD      4546
MXN      2775
SEK      1351
HKD      1203
NZD       856
DKK       843
SGD       736
CHF       666
JPY       490
NOK       466
PLN         4
Name: currency, dtype: int64

In [15]:
# df.creator = df.creator.apply(lambda x: x.replace(' "', " ").replace('" '," "))

In [16]:
# for i,creator in enumerate(df.creator):
#     print(i)
#     json.loads(creator)

## Feature Engineering

- has prior success
- rate of funding (fast funder, slow funder)
- # of words in blurb


#### Has prior success

From the creator dictionary string, the user id is pulled, and then that id is used to check if that user has more than 1 successful campaign.

In [17]:
def checkit(string):
    i = 0
    for letter in string:
        if letter.isdigit():
            i += 1
        else:
            break
    return i
df['creator_id'] = df.creator.apply(lambda x: int(x[6:6+checkit(x[6:])]))
temp = df.creator_id.value_counts()
df['has_prior_success'] = [1 if temp.loc[x] > 1 else 0 for x in df.creator_id]

In [18]:
df.has_prior_success.value_counts()

0    147580
1     44295
Name: has_prior_success, dtype: int64

#### Project length

We want to determine if the length of a project has an effect on the success of the campaign.

In [19]:
df['project_length'] = df.deadline - df.launched_at
df.project_length = df.project_length.apply(lambda x: x.days)

In [20]:
len(df.blurb[0].split())

21

#### Blurb Length

To see if the number of words in the blurb is important to the success of a project.

In [21]:
df['blurb_length'] = [len(str(x).split()) for x in df.blurb]

#### Pledge per backer

In [22]:
df.backers_count.value_counts()

0       15885
1       15434
2       10694
3        7331
4        5255
        ...  
2373        1
4422        1
6599        1
3013        1
1726        1
Name: backers_count, Length: 3410, dtype: int64

In [23]:
df['pledge_per_backer'] = np.where(df.backers_count == 0, 0, df.usd_pledged / df.backers_count)

#### Goal timedelta

In [24]:
df['goal_needed_per_day'] = df.usd_goal / df.project_length

In [25]:
df['raised_per_day'] = df.usd_pledged / df.project_length

In [26]:
df.columns

Index(['backers_count', 'blurb', 'category', 'country', 'created_at',
       'creator', 'currency', 'deadline', 'goal', 'is_starrable',
       'launched_at', 'location', 'name', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type', 'usd_goal',
       'creator_id', 'has_prior_success', 'project_length', 'blurb_length',
       'pledge_per_backer', 'goal_needed_per_day', 'raised_per_day'],
      dtype='object')

In [27]:
df.usd_type.value_counts()

domestic         146370
international     45263
Name: usd_type, dtype: int64

In [28]:
df.head()

,backers_count,blurb,category,country,created_at,creator,currency,deadline,goal,is_starrable,launched_at,location,name,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,usd_goal,creator_id,has_prior_success,project_length,blurb_length,pledge_per_backer,goal_needed_per_day,raised_per_day
0,2,I am building a website as a platform for stud...,"{""id"":362,""name"":""Makerspaces"",""slug"":""technol...",AU,2018-05-11 00:48:16,"{""id"":576665900,""name"":""Emma - Teaching Little...",AUD,2018-06-13 18:22:42,1000.0,False,2018-05-14 18:22:42,"{""id"":1102170,""name"":""Geelong"",""slug"":""geelong...",STEAM Challenges for kids,11.32,"{""id"":3378817,""project_id"":3378817,""state"":""in...",steam-challenges-for-kids,https://www.kickstarter.com/discover/categorie...,False,False,failed,2018-06-13 18:22:42,0.754091,"{""web"":{""project"":""https://www.kickstarter.com...",8.536309,domestic,754.09094,576665900,0,30,21,4.268155,25.136365,0.284544
1,43,"For my first solo museum exhibition, I'm const...","{""id"":288,""name"":""Installations"",""slug"":""art/i...",US,2017-03-28 20:09:28,"{""id"":1570512397,""name"":""Mark Reamy"",""is_regis...",USD,2017-05-03 09:44:23,3000.0,False,2017-04-03 09:44:23,"{""id"":1123319,""name"":""Gwangju-Si"",""slug"":""gwan...",Moving Mountains - An Exhibition in South Korea,3390.00,"{""id"":2940021,""project_id"":2940021,""state"":""in...",moving-mountains-an-exhibition-in-south-korea,https://www.kickstarter.com/discover/categorie...,True,True,successful,2017-05-03 09:44:24,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",3390.000000,domestic,3000.00000,1570512397,0,30,21,78.837209,100.000000,113.000000
2,243,"Jenn lives in an underground bunker, protected...","{""id"":302,""name"":""Thrillers"",""slug"":""film & vi...",US,2014-09-24 01:24:30,"{""id"":579435811,""name"":""Steve Desmond"",""is_reg...",USD,2014-11-12 21:09:45,16500.0,False,2014-10-13 21:09:45,"{""id"":2442047,""name"":""Los Angeles"",""slug"":""los...",Monsters - The Short Film,28601.11,"{""id"":1376329,""project_id"":1376329,""state"":""in...",monsters-the-short-film,https://www.kickstarter.com/discover/categorie...,True,True,successful,2014-11-12 21:09:47,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",28601.110000,domestic,16500.00000,579435811,0,30,23,117.700041,550.000000,953.370333
3,2,I want to create a book that displays my work ...,"{""id"":278,""name"":""People"",""slug"":""photography/...",US,2015-01-05 16:13:00,"{""id"":528048907,""name"":""Brandi Carroll"",""is_re...",USD,2015-02-05 22:11:46,5000.0,False,2015-01-06 22:11:46,"{""id"":2374330,""name"":""Caneyville"",""slug"":""cane...",Imagine,26.00,"{""id"":1612632,""project_id"":1612632,""state"":""in...",imagine-0,https://www.kickstarter.com/discover/categorie...,False,False,failed,2015-02-05 22:11:46,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",26.000000,domestic,5000.00000,528048907,0,30,28,13.000000,166.666667,0.866667
4,210,A series of hard enamel pins and accessories,"{""id"":26,""name"":""Crafts"",""slug"":""crafts"",""posi...",GB,2020-07-02 05:46:43,"{""id"":926607047,""name"":""Laura Croft"",""slug"":""f...",GBP,2020-08-28 12:59:06,1000.0,False,2020-07-31 12:59:06,"{""id"":26279,""name"":""Lincoln"",""slug"":""lincoln-g...",Monsters and Maw Enamel Pins,4919.00,"{""id"":4025596,""project_id"":4025596,""state"":""in...",monsters-and-maw-enamel-pins,https://www.kickstarter.com/discover/categorie...,True,False,successful,2020-08-28 12:59:06,1.298425,"{""web"":{""project"":""https://www.kickstarter.com...",6386.951345,domestic,1298.42475,926607047,1,28,8,30.414054,46.372312,228.105405


## Categorical Columns

In [29]:
dummy_cols = [
    'country',
    'is_starrable',
    'spotlight',
    'staff_pick',
    'state',
]
df = pd.get_dummies(df,
                    columns = dummy_cols,
                    drop_first = True)
df.head()

,backers_count,blurb,category,created_at,creator,currency,deadline,goal,launched_at,location,name,pledged,profile,slug,source_url,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,usd_goal,creator_id,has_prior_success,project_length,blurb_length,pledge_per_backer,goal_needed_per_day,raised_per_day,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_GR,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_PL,country_SE,country_SG,country_SI,country_US,is_starrable_True,spotlight_True,staff_pick_True,state_failed,state_live,state_successful
0,2,I am building a website as a platform for stud...,"{""id"":362,""name"":""Makerspaces"",""slug"":""technol...",2018-05-11 00:48:16,"{""id"":576665900,""name"":""Emma - Teaching Little...",AUD,2018-06-13 18:22:42,1000.0,2018-05-14 18:22:42,"{""id"":1102170,""name"":""Geelong"",""slug"":""geelong...",STEAM Challenges for kids,11.32,"{""id"":3378817,""project_id"":3378817,""state"":""in...",steam-challenges-for-kids,https://www.kickstarter.com/discover/categorie...,2018-06-13 18:22:42,0.754091,"{""web"":{""project"":""https://www.kickstarter.com...",8.536309,domestic,754.09094,576665900,0,30,21,4.268155,25.136365,0.284544,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,43,"For my first solo museum exhibition, I'm const...","{""id"":288,""name"":""Installations"",""slug"":""art/i...",2017-03-28 20:09:28,"{""id"":1570512397,""name"":""Mark Reamy"",""is_regis...",USD,2017-05-03 09:44:23,3000.0,2017-04-03 09:44:23,"{""id"":1123319,""name"":""Gwangju-Si"",""slug"":""gwan...",Moving Mountains - An Exhibition in South Korea,3390.00,"{""id"":2940021,""project_id"":2940021,""state"":""in...",moving-mountains-an-exhibition-in-south-korea,https://www.kickstarter.com/discover/categorie...,2017-05-03 09:44:24,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",3390.000000,domestic,3000.00000,1570512397,0,30,21,78.837209,100.000000,113.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1
2,243,"Jenn lives in an underground bunker, protected...","{""id"":302,""name"":""Thrillers"",""slug"":""film & vi...",2014-09-24 01:24:30,"{""id"":579435811,""name"":""Steve Desmond"",""is_reg...",USD,2014-11-12 21:09:45,16500.0,2014-10-13 21:09:45,"{""id"":2442047,""name"":""Los Angeles"",""slug"":""los...",Monsters - The Short Film,28601.11,"{""id"":1376329,""project_id"":1376329,""state"":""in...",monsters-the-short-film,https://www.kickstarter.com/discover/categorie...,2014-11-12 21:09:47,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",28601.110000,domestic,16500.00000,579435811,0,30,23,117.700041,550.000000,953.370333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1
3,2,I want to create a book that displays my work ...,"{""id"":278,""name"":""People"",""slug"":""photography/...",2015-01-05 16:13:00,"{""id"":528048907,""name"":""Brandi Carroll"",""is_re...",USD,2015-02-05 22:11:46,5000.0,2015-01-06 22:11:46,"{""id"":2374330,""name"":""Caneyville"",""slug"":""cane...",Imagine,26.00,"{""id"":1612632,""project_id"":1612632,""state"":""in...",imagine-0,https://www.kickstarter.com/discover/categorie...,2015-02-05 22:11:46,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",26.000000,domestic,5000.00000,528048907,0,30,28,13.000000,166.666667,0.866667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,210,A series of hard enamel pins and accessories,"{""id"":26,""name"":""Crafts"",""slug"":""crafts"",""posi...",2020-07-02 05:46:43,"{""id"":926607047,""name"":""Laura Croft"",""slug"":""f...",GBP,2020-08-28 12:59:06,1000.0,2020-07-31 12:59:06,"{""id"":26279,""name"":""Lincoln"",""slug"":""lincoln-g...",Monsters and Maw Enamel Pins,4919.00,"{""id"":4025596,""project_id"":4025596,""state"":""in...",monsters-and-maw-enamel-pins,https://www.kickstarter.com/discover/categorie...,2020-08-28 12:59:06,1.298425,"{""web"":{""p

In [30]:
df.columns

Index(['backers_count', 'blurb', 'category', 'created_at', 'creator',
       'currency', 'deadline', 'goal', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type', 'usd_goal',
       'creator_id', 'has_prior_success', 'project_length', 'blurb_length',
       'pledge_per_backer', 'goal_needed_per_day', 'raised_per_day',
       'country_AU', 'country_BE', 'country_CA', 'country_CH', 'country_DE',
       'country_DK', 'country_ES', 'country_FR', 'country_GB', 'country_GR',
       'country_HK', 'country_IE', 'country_IT', 'country_JP', 'country_LU',
       'country_MX', 'country_NL', 'country_NO', 'country_NZ', 'country_PL',
       'country_SE', 'country_SG', 'country_SI', 'country_US',
       'is_starrable_True', 'spotlight_True', 'staff_pick_True',
       'state_failed', 'state_live', 'state_successful'],
      dtype='object')

# Train Test Split

The dataframe is randomly separated into training/test data in a 80:20 split. The model will be fit on the training set, then then used to predict the target of the test set.  A random seed value is included for preproducibility.

In [80]:
model_features = [
#     'backers_count',
#     'usd_pledged',
    'usd_goal',
    'project_length',
    'blurb_length',
    'has_prior_success',
#     'pledge_per_backer', 
    'goal_needed_per_day', 
#     'raised_per_day',
       'country_AU', 'country_BE', 'country_CA', 'country_CH', 'country_DE',
       'country_DK', 'country_ES', 'country_FR', 'country_GB', 'country_GR',
       'country_HK', 'country_IE', 'country_IT', 'country_JP', 'country_LU',
       'country_MX', 'country_NL', 'country_NO', 'country_NZ', 'country_PL',
       'country_SE', 'country_SG', 'country_SI', 'country_US',
    'is_starrable_True',
#     'spotlight_True',
    'staff_pick_True',
    'state_successful'
]
model_df = df[model_features]

In [81]:
X = model_df.drop(columns = 'state_successful')
y = model_df.state_successful
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=69
)

## Random Forest Gridsearch

In [86]:
params = {
    'n_estimators' : [100],
    'max_features': ['auto'],
    'criterion' : ['gini'],
    'max_depth' : [15],
    'min_samples_leaf' : [1],
    'min_samples_split' : [2],
    'bootstrap' : ['True']
}
forest = RandomForestClassifier(
    class_weight = 'balanced',
    random_state = 69,
    n_jobs = -1,
)
forestGS = GridSearchCV(
    estimator = forest,
    param_grid = params,
    cv = 5,
    scoring = 'f1',
    verbose = 1,
    n_jobs = -1
)
forestGS.fit(X_train,
          y_train)
forestGS.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.6s finished


RandomForestClassifier(bootstrap='True', class_weight='balanced',
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=69, verbose=0, warm_start=False)

In [87]:
y_pred = forestGS.best_estimator_.predict(X_test)
f1_score(y_test,
         y_pred,
)

0.7222541966426859

In [88]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,
               y_pred)

0.6981889250814333

In [85]:
pd.DataFrame(forestGS.best_estimator_.feature_importances_,
             index = X.columns)

,0
usd_goal,0.220996
project_length,0.098664
blurb_length,0.012725
has_prior_success,0.115025
goal_needed_per_day,0.114564
country_AU,0.000577
country_BE,0.000315
country_CA,0.000420
country_CH,0.000158
country_DE,0.001745


In [ ]:
f1_score(y_train, forestGS.best_estimator_.predict(X_train))

In [ ]:
f1_score(y_test, forestGS.best_estimator_.predict(X_test))